### Machine Learning: TF Audio Classification Model Creation and Training Evaluation Document
In this document we will read in our feature npy files, construct, train and evaluate a model for audio classification:

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import librosa
from netplot import ModelPlot
from tensorflow.keras.utils import plot_model
print(tf.__version__)

2.7.0
